#Libraries

In [ ]:
import tensorflow as tf
import numpy as np
from google.colab import drive

# Dataset in details

In [ ]:
drive.mount('/content/drive')
dataset = open('/content/drive/MyDrive/Persian-WikiText-1.txt', 'rb').read().decode(encoding='UTF-8')
dataset = dataset.replace('\n', '') #put all lines one after the other
sample = dataset[:1000000]#since our data is too large we have to take a sample from it and work with that
char_data = sorted(set(sample)) #get a list of unique characters of the text
len(char_data)  #number of unique characters

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


224

In [ ]:
#using this we are generating an index mapping for the characters in the text
charTOindex = {u:i for i, u in enumerate(char_data)}
indexTOchar = np.array(char_data)
first_30 = dict(list(charTOindex.items())[:30])
txtTOint = np.array([charTOindex[c] for c in sample])
final_chars = tf.data.Dataset.from_tensor_slices(txtTOint)
total_indexes = []
for i in final_chars.take(5): total_indexes.append(i)

In [ ]:
#now we have to split our data into batches for training
batches = final_chars.batch(100, drop_remainder=True)

In [ ]:
#here i define a function for splitting each data sample into target and input
def sit(batch):
    input_text = batch[:-1]
    target_text = batch[1:]
    return input_text, target_text

In [ ]:
final_dataset = batches.map(sit)
total_indexes = []
for i in final_dataset.take(5) : total_indexes.append(i[0].numpy())
final_dataset = final_dataset.batch(50, drop_remainder=True)
x_train, y_train =next(iter(final_dataset))
x_train

<tf.Tensor: shape=(50, 99), dtype=int64, numpy=
array([[167, 174, 176, ...,   0, 150, 175],
       [163, 176, 159, ..., 192, 149, 159],
       [  0, 149, 170, ..., 150, 195, 174],
       ...,
       [176,   0, 170, ..., 195, 152,   0],
       [149, 195,   0, ...,   0, 152, 149],
       [189, 161,   0, ..., 157,   0, 192]])>

#implement model

In [ ]:
def loss_f(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(len(char_data), 256),
    tf.keras.layers.GRU(1024, return_sequences=True),
    tf.keras.layers.Dense(224)
])
model.compile(optimizer='adam', loss=loss_f)
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath='gru_prediction_model.h5', save_weights_only=True)
history = model.fit(final_dataset, epochs=30, callbacks=[checkpoint])

Epoch 1/30
200/200 [==============================] - 16s 66ms/step - loss: 2.8729
Epoch 2/30
200/200 [==============================] - 9s 46ms/step - loss: 2.3686
Epoch 3/30
200/200 [==============================] - 9s 46ms/step - loss: 2.1412
Epoch 4/30
200/200 [==============================] - 9s 47ms/step - loss: 1.9524
Epoch 5/30
200/200 [==============================] - 10s 48ms/step - loss: 1.8113
Epoch 6/30
200/200 [==============================] - 10s 48ms/step - loss: 1.7041
Epoch 7/30
200/200 [==============================] - 10s 50ms/step - loss: 1.6177
Epoch 8/30
200/200 [==============================] - 10s 51ms/step - loss: 1.5411
Epoch 9/30
200/200 [==============================] - 10s 51ms/step - loss: 1.4676
Epoch 10/30
200/200 [==============================] - 10s 49ms/step - loss: 1.3929
Epoch 11/30
200/200 [==============================] - 10s 49ms/step - loss: 1.3134
Epoch 12/30
200/200 [==============================] - 10s 48ms/step - loss: 1.2282
Epoc

In [ ]:
model.save('prediction_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
model2 = tf.keras.Sequential([
    tf.keras.layers.Embedding(224, 256),
    tf.keras.layers.GRU(1024, return_sequences=True),
    tf.keras.layers.Dense(224)
])
model2.load_weights('prediction_model.h5')
model2.build(tf.TensorShape([1, None]))
model2.reset_states()

In [ ]:
def predict_characters(base_model, char_num, starting_text):
    input_eval = [charTOindex[s] for s in starting_text]
    input_eval = tf.expand_dims(input_eval, 0)
    text_generated = [starting_text]
    for i in range(char_num):
        predictions = base_model.predict(input_eval, verbose=0)
        predictions = tf.squeeze(predictions, 0)
        predicted_ids = np.array(predictions.numpy()).argmax(axis=1).reshape(-1, 1)[-1][0]
        message = np.append(input_eval[0].numpy(), predicted_ids)[1:]
        input_eval = tf.expand_dims(message, 0)
        text_generated.append(indexTOchar[predicted_ids])

    for i in ''.join(text_generated).split('\n'):
        print(i)

In [ ]:
predict_characters(model2,50, "سلام")

سلامی باشد. بنام های سرزمین سوزان امروزی و فرمان به سو
